In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import pdb #데이터가 크고 해상도가 크면 속도가 걸릴 것. 그래서 중간중간에 테스트하면서 돌려야 하는데 for문 안에서 동작하는 것을 하나하나 보려면 디버깅 툴이 필요

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #GPU를 cuda라고 함
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') #cuda:0하면 GPU 서버 중 0번째꺼 써라. 위의 것은 GPU 서버가 하나든 여러개든 다 쓰겠다는 것.
device

In [ ]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='../../data/', 
                                           train = True,
                                           transform = transforms.ToTensor(),
                                           download = True)

test_dataset = torchvision.datasets.MNIST(root='../../data/', 
                                           train = False,
                                           transform = transforms.ToTensor())

train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               shuffle = True)

test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=batch_size,
                                               shuffle = False)

In [ ]:
class ConvNet(nn.Module):
  def __init__(self, num_classes=10):
    super(ConvNet, self).__init__()
    #레이어 하나 만들기
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2), 
        #입력채널, 출력채널(hidden layer의 필터 개수), 필터 개수
        #원본사이즈, 출력사이즈 동일하게 가져가려면 padding 써야 함. 5x5일 때 stride=1, padding=2

        nn.ReLU())
    
    self.layer2 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.layer3 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2), 
        nn.ReLU())
    
    self.layer4 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.layer5 = nn.Linear(7*7*32, num_classes) #입력, 출력
  
  def forward(self, x):
    #pdb.set_trace()
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = out.reshape(out.size(0), -1) #2차원을 1차원으로 펼치는 과정
    out = self.layer5(out)
    return out


In [ ]:
model = ConvNet(num_classes).to(device)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_step = len(train_dataloader) #600번
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_dataloader):
    images = images.to(device)
    labels = labels.to(device)

    #1.
    #pdb.set_trace() #위에서 멈췄기 때문에 다음은 이걸 실행할 차례야.
    preds=model(images)
    loss = loss_function(preds, labels)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    if((i+1)%100==0):
      print('Epoch [{}/{}], Step [{}/{}], Loss:{:.3f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# 코드 돌리면 [1/5]가 1 Epoch

In [ ]:
model.eval() #모델을 eval()
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_dataloader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images) #위에 정의해놓은 forward()가 호출된다.

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print('Test Accuracy : {}%'.format(100 * correct / total))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
images.shape 
# 항상 4차원으로 나옴. 1, 28, 28이 한장의 이미지인데, 배치 사이즈가 100이므로 이렇게 4차원...

In [ ]:
images[0].size()
#채널을 제외한 0번째 이미지의 사이즈

In [ ]:
images[0,0].size()
#0번째 이미지의 사이즈

In [ ]:
images[1].size()
#1번째 이미지의 사이즈

In [ ]:
labels.size()

In [ ]:
for idx in range(30):
  image = np.array(images[idx, 0].detach().cpu())
  label = labels[idx].item()
  pred = predicted[idx].item()
  plt.figure(figsize=(2,2))
  plt.title(f'Label : {label} Predict : {pred}')
  plt.imshow(image * 255)